<a href="https://colab.research.google.com/github/StuartLiv/CPSC-440-Project/blob/main/models/hooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.transforms.functional as F

In [2]:
import os
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array # TODO don't use keras
from keras.utils import load_img
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import random

from google.colab import userdata

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


# 1. Set up project in runtime
Copy the following cell and run it with your credentials to clone the repo, getting datasets as runtime files, and then delete your credentials.

In [3]:
!git config --global user.email {userdata.get('email')}
!git config --global user.name {userdata.get('name')}
!git clone https://{userdata.get('token')}@github.com/StuartLiv/CPSC-440-Project
%pwd

fatal: destination path 'CPSC-440-Project' already exists and is not an empty directory.


'/content'

# 2. Preprocess data

Datasets:
- `medset_multisize`
  - Single datasets. Dataset construction code in repo too
  - 12000 Images, in randomized order
  - Res 256x256 (grayscale images also available in 128x128, 64x64, 32x32)


In [4]:
''' Returns an array containing all images in folder, sorted by filename. Useful for color/gray/downsize versions in different folders with same filename.
@param path - directory with images
@return
'''
def make_image_arr(path):

  def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

  files = os.listdir(path)
  files = sorted_alphanumeric(files)

  arr = []
  for i in tqdm(files):
    img = load_img(path + '/'+i)
    arr.append(img_to_array(img) / 255)

  return np.array(arr)


Load datasets, partition sets, and make tensors in color-layer order for convolutions



In [5]:
training_proportion = 0.8
validation_proportion = 0.0

def load_and_partition(path):
  arr = make_image_arr(path)
  cutoff = int(len(arr)*training_proportion)
  return arr[:cutoff], arr[cutoff:]

def make_tensor(arr):
  #Axis transformation: Shape (n, h, w, 3) -> (n, 3, h, w)
  #Inversion is np.moveaxis(arr, [2,2], [1,1])
  return torch.from_numpy(np.moveaxis(arr, [3,1], [1,2]))

#Extract single channel, since all 3 are the same in gray images
def make_gray_tensor(arr):
  return make_tensor(arr)[:, 0:1, :, :]

gray_train, gray_test = load_and_partition('/content/CPSC-440-Project/datasets/medset_multisize/gray32')

gray_train_tensors = make_gray_tensor(gray_train)
gray_test_tensors = make_gray_tensor(gray_test)

100%|██████████| 12000/12000 [00:02<00:00, 5753.71it/s]


# 3. Model
Model generated below:

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 3, 3, padding=1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Create the autoencoder
autoencoder = Autoencoder()

Train Model (SGD)

In [7]:
autoencoder.encoder = torch.load('/content/CPSC-440-Project/models/32_dim_encoder_weights')
autoencoder.decoder = torch.load('/content/CPSC-440-Project/models/decoder_weights_256')
autoencoder.eval()


Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track

## Hook
Add hooks to self.encoder or self.decoder, with the indices listed by `.eval` above!

In [8]:

# a dict to store the activations
activation = {}
def getActivation(name):
  # the hook signature
  def hook(model, input, output):
    activation[name] = output.detach().cpu().numpy()
  return hook

# register forward hooks on the layers with the following dims:
dims = [32,64,128,256]
autoencoder.encoder[-1].register_forward_hook(getActivation('en'))
autoencoder.decoder[0].register_forward_hook(getActivation('de64'))
autoencoder.decoder[6].register_forward_hook(getActivation('de128'))
autoencoder.decoder[12].register_forward_hook(getActivation('de256'))

# h3 = autoencoder.decoder.register_forward_hook(getActivation('sigmoid')) # this is excatly the output!


In [9]:
def get_rank(S,cut):
  sum = np.sum(S)
  i = 0
  while np.sum(S[:i]) < cut * np.sum(S[i:]):
    i += 1
  return i

In [ ]:
n = 100
ranks = np.zeros((3,n))
count = 0
for i in tqdm(np.random.randint(0, len(gray_test), n)):
  # Colorize the image
  t = gray_test_tensors[i].unsqueeze(0).to(device)
  autoencoder(t)

  for j in range(3):
    key = list(activation.keys())[j]
    dim = dims[j]
    print(activation[key][0].shape)
    U,S,V = np.linalg.svd(activation[key][0].reshape(-1,dim**2))
    print(S.shape)
    ranks[j,count] = get_rank(S,3)

  count += 1

  0%|          | 0/100 [00:00<?, ?it/s]

(256, 32, 32)
(256,)
(128, 64, 64)
(128,)
(32, 128, 128)


  1%|          | 1/100 [00:09<15:59,  9.69s/it]

(32,)
(256, 32, 32)
(256,)
(128, 64, 64)
(128,)
(32, 128, 128)


  2%|▏         | 2/100 [00:19<16:00,  9.80s/it]

...

(32,)
(256, 32, 32)
(256,)
(128, 64, 64)
(128,)
(32, 128, 128)


  9%|▉         | 9/100 [01:25<14:20,  9.45s/it]

(32,)
(256, 32, 32)
(256,)
(128, 64, 64)
(128,)
(32, 128, 128)


 10%|█         | 10/100 [01:35<14:21,  9.57s/it]

(32,)
(256, 32, 32)
(256,)
(128, 64, 64)
(128,)
(32, 128, 128)


In [14]:
ranks

array([[ 6.],
       [ 5.],
       [10.]])

In [ ]:
np.linalg.svd(activation['de256'][0].reshape(8,256**2)[:4])

In [ ]:
# rank of convolutions
# out of max 16, 64, 256
np.mean(ranks, axis=1)

In [ ]:
np.save('ranks.npy',ranks)

# 4. Push changes

Look for any changes in the project directory, excluding this notebook, and push them.


In [ ]:
cd CPSC-440-Project

# 4. Save notebook

Commiting this notebook requires a special maneuver:

> File > Save a copy in github > enter original `path` + new commit msg

That's it!